# Loop 1 LB Feedback Analysis

**Baseline submitted:** CV 70.6473 | LB 70.6473 (gap: 0.0000)

This is a perfect CV-LB alignment, which is expected for an optimization problem where the score is deterministic.

**Target:** 68.894234
**Gap to beat:** 1.753 points (2.5%)

In [ ]:
import pandas as pd
import numpy as np
import json

# Load baseline metrics
with open('/home/code/experiments/001_baseline/metrics.json', 'r') as f:
    baseline_metrics = json.load(f)

print(f"Baseline CV Score: {baseline_metrics['cv_score']:.6f}")
print(f"Target: {baseline_metrics['target']}")
print(f"Gap: {baseline_metrics['gap_to_target']:.6f}")

In [ ]:
# Analyze per-N scores from the snapshot metrics
import os

# Load the detailed per-N scores from the snapshot
snapshot_metrics_path = '/home/nonroot/snapshots/santa-2025/21328309254/code/experiments/001_baseline/metrics.json'
with open(snapshot_metrics_path, 'r') as f:
    detailed_metrics = json.load(f)

scores_by_n = detailed_metrics['scores_by_n']

# Convert to DataFrame for analysis
scores_df = pd.DataFrame([
    {'n': int(k), 'score': v} for k, v in scores_by_n.items()
]).sort_values('n')

print(f"Total N values: {len(scores_df)}")
print(f"\nTop 20 worst N values (highest score contribution):")
worst_n = scores_df.nlargest(20, 'score')
for _, row in worst_n.iterrows():
    print(f"  N={int(row['n']):3d}: {row['score']:.6f}")

In [ ]:
# Calculate cumulative score contribution
scores_df = scores_df.sort_values('n')
scores_df['cumulative'] = scores_df['score'].cumsum()
scores_df['pct_of_total'] = 100 * scores_df['cumulative'] / scores_df['score'].sum()

print("\nCumulative score contribution:")
for n in [10, 20, 30, 50, 100, 150, 200]:
    row = scores_df[scores_df['n'] == n].iloc[0]
    print(f"  N=1-{n}: {row['cumulative']:.4f} ({row['pct_of_total']:.1f}% of total)")

In [ ]:
# Key insight: How much improvement is needed per N range?
target = 68.894234
current = baseline_metrics['cv_score']
gap = current - target

print(f"\nTotal gap to close: {gap:.6f}")
print(f"\nIf we improve each N by the same percentage:")
required_pct = 100 * (1 - target/current)
print(f"  Required improvement: {required_pct:.2f}%")

print(f"\nIf we focus on small N (1-20):")
small_n_score = scores_df[scores_df['n'] <= 20]['score'].sum()
print(f"  Current small N score: {small_n_score:.4f}")
print(f"  If we reduce small N by 50%: saves {small_n_score * 0.5:.4f}")
print(f"  If we reduce small N by 30%: saves {small_n_score * 0.3:.4f}")

In [ ]:
# Compare with theoretical minimum
# For a single tree (N=1), the minimum bounding box is the tree's own bounding box
# Tree dimensions: width ~0.7, height ~1.0 (from -0.2 to 0.8)
# At 45 degrees rotation, the bounding box is minimized

import math

# Tree vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def get_bbox_at_angle(angle_deg):
    angle_rad = math.radians(angle_deg)
    cos_a = math.cos(angle_rad)
    sin_a = math.sin(angle_rad)
    
    rotated_x = [cos_a * x - sin_a * y for x, y in zip(TX, TY)]
    rotated_y = [sin_a * x + cos_a * y for x, y in zip(TX, TY)]
    
    width = max(rotated_x) - min(rotated_x)
    height = max(rotated_y) - min(rotated_y)
    return max(width, height)

# Find optimal angle for N=1
best_angle = 0
best_side = float('inf')
for angle in range(0, 360, 1):
    side = get_bbox_at_angle(angle)
    if side < best_side:
        best_side = side
        best_angle = angle

print(f"N=1 optimal angle: {best_angle}° with side={best_side:.6f}")
print(f"N=1 optimal score: {best_side**2:.6f}")
print(f"N=1 current score: {scores_df[scores_df['n'] == 1]['score'].values[0]:.6f}")

In [ ]:
# Summary of findings
print("="*60)
print("KEY FINDINGS FOR STRATEGY")
print("="*60)
print(f"\n1. CV-LB alignment is PERFECT (gap = 0.0000)")
print(f"   This means CV improvements will directly translate to LB improvements.")
print(f"\n2. Gap to target: {gap:.4f} points ({required_pct:.2f}%)")
print(f"\n3. Small N (1-20) contribute {small_n_score:.4f} points ({100*small_n_score/current:.1f}% of total)")
print(f"   These are the highest-leverage targets.")
print(f"\n4. N=1 is already at optimal (45° rotation)")
print(f"   Score: {scores_df[scores_df['n'] == 1]['score'].values[0]:.6f}")
print(f"\n5. Top improvement opportunities:")
for _, row in worst_n.head(10).iterrows():
    n = int(row['n'])
    score = row['score']
    # Estimate potential improvement (assume 10% possible)
    potential = score * 0.1
    print(f"   N={n:3d}: current={score:.4f}, potential savings={potential:.4f}")